### Import package：

In [19]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import time
import os
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import pandas as pd

### Load data：

In [20]:
rfilename=".\example.csv"
df=pd.read_csv(rfilename)
df.tail() #data showd as follow:

,peptide,smile
0,ACEF,C[C@H](N)C(=O)N[C@@H](CS)C(=O)N[C@@H](CCC(=O)O...
1,TCTPW,C[C@@H](O)[C@H](N)C(=O)N[C@@H](CS)C(=O)N[C@H](...
2,AGCF,C[C@H](N)C(=O)NCC(=O)N[C@@H](CS)C(=O)N[C@@H](C...
3,APDCR,C[C@H](N)C(=O)N1CCC[C@H]1C(=O)N[C@@H](CC(=O)O)...


In [21]:
peptideseq=list(df["peptide"])
peptidesmi=list(df["smile"])

### Some Function:

In [22]:
def openpage():
    """
    open the browser and enter the page
    return driver
    """
    driver=webdriver.Firefox() #open the brower
    driver.implicitly_wait(10)
    driver.set_window_size(580,840)
    driver.get("https://admetlab3.scbdd.com/server/screening") #open the page
    return driver


def dataprepare(peptidesmi,peptideseq):
    """
    prepare the input data
    return inputdata,name and state
    """
    inputdata=[]
    for i in range(1):
        try:
            inputdata.append(peptidesmi.pop())
            name = peptideseq.pop()
            state = True
        except:
            state = False
    return inputdata,name,state

def inputsmiles(driver,inputdata,name):
    """
    according to the length of sequence to set the waitting time
    and submit the smiles of peptide
    return waittime, a int object
    """
    workcount=len(inputdata)
    waittime = len(name)*2  #set the waitting time
    for i in range(workcount):
        inputtext= inputdata[i]+"\n"     
    listinput_button = "/html/body/div/div/main/div/ul/li[2]/button"
    text_area_xpath = "/html/body/div/div/main/div/div/div[2]/form/div[1]/textarea"
    submit_button = "/html/body/div/div/main/div/div/div[2]/form/div[2]/div[4]/button"
    driver.find_element(By.XPATH,listinput_button).click()
    driver.find_element(By.XPATH,text_area_xpath).send_keys(inputtext)  #send the smiles
    submit_button_click=driver.find_element(By.XPATH,submit_button).click() #click the submit button
    return waittime

def WaittingAndDownload(driver,state,inputdata,name,dlpath="C:\\Users\\ff\\Downloads\\",newpath="C:\\Users\\ff\\admetlab\\"):
    """
    download the result and modify the stored point.
    Note！ Files can only be moved within the same disk!
    dlpath was the default download point of bowser.
    newpath was the point want to stored data.
    return None
    
    """
    waittime = inputsmiles(driver,inputdata,name)
    time.sleep(waittime)
    looptime = 0
    while state:
        looptime+=1
        try:
            downloadlink = "/html/body/div/div/div/div[1]/div[3]/button"
            driver.find_element(By.XPATH,downloadlink).click() #successfully download result
            state=False
        except:
            print("waitting for twice")
            time.sleep(waittime) #waitting for 2 min again
            if looptime == 2:
                state=False  #give up this work
    time.sleep(5) #waitting for result download
    files=os.listdir(dlpath)  # move the file to new saving point
    try:
        for f in files:
            if f[0:3]=="tmp":
                os.rename(dlpath+f,newpath+name+".csv")
                break
    except:
        for f in files:
            if f[0:3]=="tmp":
                try:
                    os.remove(f)
                except:
                    pass  
    driver.back()  #go back to last page
    time.sleep(3)
    listinput_button = "/html/body/div/div/main/div/ul/li[2]/button"
    driver.find_element(By.XPATH,listinput_button).click()
    reset_button = "/html/body/div/div/main/div/div/div[2]/form/div[2]/div[1]/button"
    driver.find_element(By.XPATH,reset_button).click()
    return None

def tryfinishAwork(driver,peptidesmi,peptideseq,dlpath,newpath):
    inputdata,name,state=dataprepare(peptidesmi,peptideseq)
    
    WaittingAndDownload(driver,state,inputdata,name,dlpath=dlpath,newpath=newpath)
    print("Oh, "+name+" is finished.")
    return None

def loop(peptidesmi,peptideseq,dlpath,newpath):
    #initial the page:
    driver=openpage()
    print("default download point is "+dlpath)
    print("data was stored in point "+newpath)
    while len(peptidesmi) !=  0:
        tryfinishAwork(driver,peptidesmi,peptideseq,dlpath,newpath)
    return None

def cleartempfiles(dlpath):
    """
    delete the tmp file
    modify your dlpath
    return none
    """
    print("tmp file in "+dlpath+" will be removed." )
    files = os.listdir(dlpath)
    for file in files:
        if file[0:3]=="tmp":
            try:
                os.remove(dlpath+file)
                print("Note! "+file+" was removed.")
            except exception as e:
                print(e)
    return None

### Execute the task:

In [23]:
if __name__ == "__main__":
    loop(peptidesmi,peptideseq,"C:\\Users\\ff\\Downloads\\","C:\\Users\\ff\\")

default download point is C:\Users\ff\Downloads\
data was stored in point C:\Users\ff\
Oh, APDCR is finished.
Oh, AGCF is finished.
Oh, TCTPW is finished.
Oh, ACEF is finished.


In [11]:
len(peptideseq)

0